## Figure 2e: Drug combination associations with host and microbiome

This script prepares input for Figure 2e: Mediation analysis of the host and microbiome features for the selected drug combinations. Drug-feature effect graph representing potential feature mediation effects between host and microbiome features. Solid lines represent drug effects on the feature, colour represents direction of the effect. Dashed lines between features indicate potential mediation (general mediation model one-sided P  < 0.1 ), colour represents the sign of Pearson’s correlation coefficient (P < 0.1). 

For visualization purposes, the edges between drugs and features in this graph have the value of drug combination effect size for the corresponding combination. (E.g. if combination of statin wih calcium antagonist synergistically decrease VLDL Cholesterol, both statin and calcium antagonist have an edge connecting them with VLDL cholesterol, which has the edge value of the combination effect size). 

Required file in the _input_data_ folder:

- Supplementary_Table_6_2019-09-13434.xlsx
- Supplementary_Table_8_2019-09-13434.xlsx
- Supplementary_Table_10_2019-09-13434.xlsx

Figure is based on the data from Supplementary Tables 6, 8 and 10. 

In [112]:
# load required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
%matplotlib inline

In [113]:
# read combination feature mediation file
fileFolder = './input_data/'
fileName = 'Supplementary_Table_10_2019-09-13434.xlsx'
sheetName = 'Drug combinations'

mediation_df_filtered_combined_filtered = pd.read_excel(fileFolder + fileName,
                           sheet_name = sheetName)

Get feature infromation for drug and drug combination associations for further filtering

In [114]:
# read single drug features files
fileName = 'Supplementary_Table_6_2019-09-13434.xlsx'
sheetName = 'Drug group effect'
drugEffect = pd.read_excel(fileFolder + fileName,
                           sheet_name = sheetName)

In [115]:
# read drug combination features files
fileName = 'Supplementary_Table_8_2019-09-13434.xlsx'
sheetName = 'Data'
drugCombinationEffect = pd.read_excel(fileFolder + fileName,
                           sheet_name = sheetName)

In [116]:
# concatenate single drug effects and combination effects
allEffects_df = pd.concat([drugCombinationEffect, drugEffect])

In [117]:
# get the patient groups (Sample set column)
listconditions = list(set(allEffects_df['Sample set']))

For each mediation pair, get effect sizes of drugs and combinations and diseases

In [118]:
sample_set_idx = 2 # select Group T2D (3)
curset = listconditions[sample_set_idx]
# copy all effects for one condition to reduce number
selectedEffectd_df = allEffects_df[(allEffects_df['Sample set'].str.find(curset)>=0)]

In [119]:
# select only features with congruence opposite of disease (drug and disease effects are opposite)
selectedEffectd_df = selectedEffectd_df[selectedEffectd_df['Congruence']=='Opposite'].copy()

In [120]:
# create a dataframe containing information on the potential features and mediators
mediation_df_filtered_combined_filtered_features = []

for i in range(len(mediation_df_filtered_combined_filtered)):
    curfeature1 = mediation_df_filtered_combined_filtered['Feature1'].iloc[i]
    curfeature2 = mediation_df_filtered_combined_filtered['Feature2_med'].iloc[i]
    curspace1 = mediation_df_filtered_combined_filtered['FeatureSpace1'].iloc[i]
    curspace2 = mediation_df_filtered_combined_filtered['FeatureSpace2'].iloc[i]
    cureffector = mediation_df_filtered_combined_filtered['Effector'].iloc[i]

    cureffector_drugs = cureffector.replace('Combination: ', '')
    cureffector_drugs = cureffector_drugs.split(', ')

    cureffect_df = selectedEffectd_df[
                (((selectedEffectd_df['Feature display name']==curfeature1) &
                  (selectedEffectd_df['Feature space']==curspace1)) |
                 ((selectedEffectd_df['Feature display name']==curfeature2) &
                  (selectedEffectd_df['Feature space']==curspace2)))&
                ((selectedEffectd_df['Effector']=='Group contrast') |
                 (selectedEffectd_df['Effector']==cureffector) |
                 (selectedEffectd_df['Effector']==cureffector_drugs[0]) |
                 (selectedEffectd_df['Effector']==cureffector_drugs[1]))]
    if len(mediation_df_filtered_combined_filtered_features)==0:
        mediation_df_filtered_combined_filtered_features = cureffect_df.copy()
    else:
        mediation_df_filtered_combined_filtered_features = pd.concat([
            mediation_df_filtered_combined_filtered_features, cureffect_df])

In [121]:
mediation_df_filtered_combined_filtered_features_subset = \
    mediation_df_filtered_combined_filtered.copy()

Select statin, metformin, aspirin and calcium antagonists to extract the mediation graph. 

In [122]:
ploteffectors = list(set(mediation_df_filtered_combined_filtered_features_subset['Effector']))

In [123]:
ploteffectors = [item for item in ploteffectors 
                 if (('Statin' in item) & ('Metformin' in item)) |
                    (('Statin' in item) & ('Aspirin' in item)) |
                    (('Statin' in item) & ('Calcium' in item))]
ploteffectors

['Combination: Statin intake, Aspirine intake',
 'Combination: Metformin intake, Statin intake',
 'Combination: Calcium antagonist intake, Statin intake']

Filter mediation results by effectors and patient type

In [124]:
mediation_df_filtered_combined_filtered_features_subset = \
    mediation_df_filtered_combined_filtered_features_subset[
    (mediation_df_filtered_combined_filtered_features_subset['Effector']==ploteffectors[0]) |
    (mediation_df_filtered_combined_filtered_features_subset['Effector']==ploteffectors[1]) |
    (mediation_df_filtered_combined_filtered_features_subset['Effector']==ploteffectors[2])].copy()

In [125]:
feature_set = 'T2D (3)'
mediation_df_filtered_combined_filtered_features_subset = \
    mediation_df_filtered_combined_filtered_features_subset[
    mediation_df_filtered_combined_filtered_features_subset['Sample set']==feature_set].copy()

In [126]:
mediation_df_filtered_combined_filtered_features_subset = \
    mediation_df_filtered_combined_filtered_features_subset[
    mediation_df_filtered_combined_filtered_features_subset['ACME (average)_P-value']<=0.1].copy()

Select feature pairs that are present in drug combo or single drugs feature matrices

In [127]:
newcolumn_names = ['Feature1_drug1','Feature1_drug2',
                   'Feature1_drug1effect','Feature1_drug2effect',
                   'Feature1_drugcombo',
                   'Feature2_drug1','Feature2_drug2',
                   'Feature2_drug1effect','Feature2_drug2effect',
                   'Feature2_drugcombo']
for col in newcolumn_names:
    mediation_df_filtered_combined_filtered_features_subset[col] = np.zeros(
        [len(mediation_df_filtered_combined_filtered_features_subset),1])

In [128]:
# prepare a subset of features to record effect of each drug on each feature
mediation_df_filtered_combined_filtered_features_subset = \
    mediation_df_filtered_combined_filtered_features_subset.reset_index()

In [129]:
# populate the dataframe with information on which drugs are associated with each feature
for i in range(len(mediation_df_filtered_combined_filtered_features_subset)):
    curfeatures = [mediation_df_filtered_combined_filtered_features_subset['Feature1'].iloc[i],
                   mediation_df_filtered_combined_filtered_features_subset['Feature2_med'].iloc[i]]
    curspaces = [mediation_df_filtered_combined_filtered_features_subset['FeatureSpace1'].iloc[i],
                   mediation_df_filtered_combined_filtered_features_subset['FeatureSpace2'].iloc[i]]
    cureffector = mediation_df_filtered_combined_filtered_features_subset['Effector'].iloc[i]

    cureffector_drugs = cureffector.replace('Combination: ', '')
    cureffector_drugs = cureffector_drugs.split(', ')

    for curfeat_i in range(len(curfeatures)):
            for curdrug_i in range(len(cureffector_drugs)):
                cureffect_df = selectedEffectd_df[
                    (selectedEffectd_df['Feature display name']==curfeatures[curfeat_i]) &
                    (selectedEffectd_df['Feature space']==curspaces[curfeat_i]) &
                    (selectedEffectd_df['Effector']==cureffector_drugs[curdrug_i])]
                if len(cureffect_df)>0:
                    mediation_df_filtered_combined_filtered_features_subset.loc[i,
                        'Feature'+str(curfeat_i+1)+'_drug'+str(curdrug_i+1)] = \
                    cureffector_drugs[curdrug_i]
                    mediation_df_filtered_combined_filtered_features_subset.loc[i,
                        'Feature'+str(curfeat_i+1)+'_drug'+str(curdrug_i+1)+'effect'] = \
                    cureffect_df['Effect size'].values[0]
            cureffect_df = selectedEffectd_df[
                    (selectedEffectd_df['Feature display name']==curfeatures[curfeat_i]) &
                    (selectedEffectd_df['Feature space']==curspaces[curfeat_i]) &
                    (selectedEffectd_df['Effector'].str.find(cureffector_drugs[0])>=0) &
                    (selectedEffectd_df['Effector'].str.find(cureffector_drugs[1])>=0)]
            if len(cureffect_df)>0:
                mediation_df_filtered_combined_filtered_features_subset.loc[i,
                    'Feature'+str(curfeat_i+1)+'_drugcombo'] = \
                cureffect_df['Effect size'].values[0]

In [130]:
# for plotting, select only mediated features that are both affected by the drug combination
plotdata_both = mediation_df_filtered_combined_filtered_features_subset[
   (mediation_df_filtered_combined_filtered_features_subset['Feature1_drugcombo']!=0) &
   (mediation_df_filtered_combined_filtered_features_subset['Feature2_drugcombo']!=0)].copy()

In [131]:
# select only features that pass correlation p-value threshold of 0.1
plotdata_both_corr = plotdata_both[plotdata_both['FeatureFeatureCorrP']<=0.1].copy()

Save mediator features to graph

In [132]:
plotdata = plotdata_both_corr.copy()

In [133]:
# represent the data as graph nodes (drugs and features)
# connected by edges (effect sizes and correlations)
graphnodes = []
graphsources = []
graphedges = []
graphedgesP = []
nodesnames = []
nodestypes=[]
edgetype=[]
for i in range(len(plotdata)):
    cureffector = plotdata['Effector'].iloc[i]
    cureffector_drugs = cureffector.replace('Combination: ', '')
    cureffector_drugs = cureffector_drugs.split(', ')
    
    # feature 1 drug 1
    graphnodes.append(cureffector_drugs[0])
    graphsources.append(plotdata['Feature1'].iloc[i])
    graphedges.append(plotdata['Feature1_drugcombo'].iloc[i])
    edgetype.append('drug_feat')
    # feature 1 drug 2
    graphnodes.append(cureffector_drugs[1])
    graphsources.append(plotdata['Feature1'].iloc[i])
    graphedges.append(plotdata['Feature1_drugcombo'].iloc[i])
    edgetype.append('drug_feat')
    # feature 2 drug 1
    graphnodes.append(cureffector_drugs[0])
    graphsources.append(plotdata['Feature2_med'].iloc[i])
    graphedges.append(plotdata['Feature2_drugcombo'].iloc[i])
    edgetype.append('drug_feat')
    # feature 2 drug 2
    graphnodes.append(cureffector_drugs[1])
    graphsources.append(plotdata['Feature2_med'].iloc[i])
    graphedges.append(plotdata['Feature2_drugcombo'].iloc[i])
    edgetype.append('drug_feat')
    # feature 1 feature 2
    graphnodes.append(plotdata['Feature1'].iloc[i])
    graphsources.append(plotdata['Feature2_med'].iloc[i])
    graphedges.append(plotdata['FeatureFeatureCorr'].iloc[i])
    graphedgesP.append(plotdata['FeatureFeatureCorrP'].iloc[i])
    edgetype.append('feat_feat')
    # nodes info
    nodesnames.append(plotdata['Feature1'].iloc[i])
    nodestypes.append('Feature')
    nodesnames.append(plotdata['Feature2_med'].iloc[i])
    nodestypes.append('Feature')
    nodesnames.append(cureffector_drugs[0])
    nodestypes.append('Drug')
    nodesnames.append(cureffector_drugs[1])
    nodestypes.append('Drug')

In [134]:
# make a dataframe of the graph nodes
graph_df = {'Node1': graphnodes,
            'Node2': graphsources,
            'EdgeValue': graphedges,
            'EdgeType': edgetype}

graph_df = pd.DataFrame(graph_df)

graph_df = graph_df.drop_duplicates()

In [135]:
graph_df

,Node1,Node2,EdgeValue,EdgeType
0,Calcium antagonist intake,Alistipes sp. (motu_linkage_group_621),0.161954,drug_feat
1,Statin intake,Alistipes sp. (motu_linkage_group_621),0.161954,drug_feat
2,Calcium antagonist intake,VLDL-3 Cholesterol mg/dL,-0.176885,drug_feat
3,Statin intake,VLDL-3 Cholesterol mg/dL,-0.176885,drug_feat
4,Alistipes sp. (motu_linkage_group_621),VLDL-3 Cholesterol mg/dL,-0.073258,feat_feat
9,VLDL-3 Cholesterol mg/dL,Alistipes sp. (motu_linkage_group_621),-0.073258,feat_feat
10,Calcium antagonist intake,VLDL Cholesterol mg/dL,-0.179195,drug_feat
11,Statin intake,VLDL Cholesterol mg/dL,-0.179195,drug_feat
14,VLDL Cholesterol mg/dL,Alistipes sp. (motu_linkage_group_621),-0.069984,feat_feat
15,Metformin intake,methanogenesis (trimethylamine degradation) (M...,0.151456,drug_feat


In [136]:
# UNCOMMENT TO PRINT GRAPH EDGES TO FILE
#graph_df.to_csv('fig2e_mediation_graph_edges.tsv', sep='\t')

In [137]:
# make a dataframe of node types
nodetype_df = pd.DataFrame({'Node': nodesnames, 'Type': nodestypes})
nodetype_df = nodetype_df.drop_duplicates()

In [138]:
nodetype_df

,Node,Type
0,Alistipes sp. (motu_linkage_group_621),Feature
1,VLDL-3 Cholesterol mg/dL,Feature
2,Calcium antagonist intake,Drug
3,Statin intake,Drug
8,VLDL Cholesterol mg/dL,Feature
12,methanogenesis (trimethylamine degradation) (M...,Feature
13,IDL Phospholipids mg/dL,Feature
14,Metformin intake,Drug
20,"Xylene degradation, xylene => methylbenzoate (...",Feature
28,"Toluene degradation, toluene => benzoate (M00538)",Feature


In [111]:
# UNCOMMENT TO PRINT NODE TYPES TO FILE
#nodetype_df.to_csv('fig2e_mediation_graph_node_types.tsv', sep='\t')